In [1]:
import pandas as pd
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
path = './data/full.csv'

# load full data
D = pd.read_csv(path, index_col=0)

In [3]:
# extract inputs/targets
target_cols = list(filter(lambda x: 'c1_c0' in x, D.columns.values))
input_cols  = list(filter(lambda x: 'c1_c0' not in x, D.columns.values))
InputDF = D[input_cols]
TargetDF = D[target_cols]

# into numpy format
InputDF = InputDF.as_matrix()
TargetDF = TargetDF.as_matrix()

In [4]:
# 6 values, from 440 symbols, over 2768 days
total_days = InputDF.shape[0]
channels = 6
tickers = InputDF.shape[1]/channels
InputDF = np.reshape(InputDF, (total_days, tickers, channels))

In [5]:
# data params
test_size = 500;
window_size = 28;


X_train = np.array([InputDF[i:i + window_size] for i in range(InputDF.shape[0] - (test_size + window_size - 1) )])
X_test = np.array([InputDF[i:i + window_size] for i in range(InputDF.shape[0] - (test_size + window_size - 1), InputDF.shape[0] - window_size + 1)])

y_train = TargetDF[window_size - 1 : -test_size]
y_test = TargetDF[-test_size:]

# data shape
print X_train.shape, y_train.shape
print X_test.shape, y_test.shape

(2241, 28, 440, 6) (2241, 440)
(500, 28, 440, 6) (500, 440)


In [23]:
# KERAS STARTS HERE
batch_size = 16
epochs = 30

# dimensions
input_shape = (window_size, tickers, channels)

In [27]:
# model definition
# ADD PADDING??
model = Sequential()
model.add(Conv2D(16, kernel_size=(5, 5), activation='relu', input_shape=input_shape))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
# model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(440))

print model.summary()

model.load_weights('weights-lowhidden.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 24, 436, 16)       2416      
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 22, 434, 32)       4640      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 11, 217, 32)       0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 11, 217, 32)       0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 76384)             0         
_________________________________________________________________
dense_15 (Dense)             (None, 512)               39109120  
_________________________________________________________________
dropout_16 (Dropout)         (None, 512)               0         
__________

In [29]:
#sgd = SGD(lr=0.03, decay=1e-6, momentum=0.9, nesterov=True)
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='mean_squared_error', optimizer=adam)

tensorboard = keras.callbacks.TensorBoard(log_dir='./logs/5')
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)

# model.fit(X_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(X_test, y_test), callbacks=[tensorboard, reduce_lr])

In [30]:
score = model.evaluate(X_test, y_test, verbose=0)

print 'Test loss:', score
# print('Test accuracy:', score[1])

Test loss: 0.000384559142636


In [175]:
predictions = model.predict(X_test)

def labeler(x):
    margin = 0.005
    labels = np.zeros(x.shape)
    labels[np.where(x > np.log(1 + margin))] = 1
    labels[np.where(x < np.log(1 - margin))] = -1
    
    return labels

returns = np.apply_along_axis(labeler,
                              axis=1,
                              arr=predictions)

# print predictions
# print '---------------------------------------------------------'
# print y_test
# print '---------------------------------------------------------'
print len(np.where(returns == 1)[1])
print len(np.where(returns == -1)[1])
print len(np.where(returns == 0)[1])

68412
4114
147474


In [185]:
n = TargetDF.flatten()
n = np.sort(n)
n = np.split(n, 5)
print n

[array([-1.14904314, -0.94280115, -0.91082526, ..., -0.01271457,
       -0.01271448, -0.01271446]), array([-0.01271441, -0.01271433, -0.01271411, ..., -0.00313783,
       -0.0031378 , -0.00313774]), array([-0.0031376 , -0.00313755, -0.00313745, ...,  0.00385198,
        0.00385199,  0.003852  ]), array([ 0.00385203,  0.00385205,  0.00385208, ...,  0.01354637,
        0.01354639,  0.01354641]), array([ 0.01354643,  0.01354654,  0.01354662, ...,  0.609564  ,
        0.65953532,  0.70279545])]


In [189]:
n = predictions.flatten()
n = np.sort(n)
print n
print n.max()
print n.min()
n = np.split(n, 5)
print n

[-0.00184591 -0.00179229 -0.00178894 ...,  0.00184795  0.00192746
  0.00197287]
0.00197287
-0.00184591
[array([ -1.84590928e-03,  -1.79228524e-03,  -1.78894028e-03, ...,
         4.56225607e-05,   4.56239723e-05,   4.56301168e-05], dtype=float32), array([  4.56726339e-05,   4.56969792e-05,   4.56987109e-05, ...,
         2.44603289e-04,   2.44604773e-04,   2.44611176e-04], dtype=float32), array([ 0.00024464,  0.00024465,  0.00024466, ...,  0.00041278,
        0.00041278,  0.00041279], dtype=float32), array([ 0.00041279,  0.0004128 ,  0.00041281, ...,  0.00063044,
        0.00063044,  0.00063044], dtype=float32), array([ 0.00063044,  0.00063044,  0.00063044, ...,  0.00184795,
        0.00192746,  0.00197287], dtype=float32)]


In [17]:
model.save_weights('weights-lowhidden.h5')